Imports

In [ ]:
import cv2
import os
from paddleocr import PaddleOCR
import pandas as pd

Read video and capture frame

In [ ]:
video_path = "../assets/成绩单.mp4"
output_dir = "../frames"
interval = 5

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

cap = cv2.VideoCapture(video_path)
frame_count = 0
saved_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    if frame_count % interval == 0:
        filename = os.path.join(output_dir, f"frame_{saved_count:03d}.jpg")
        cv2.imwrite(filename, frame)
        print(f"Saved {filename}")
        saved_count += 1

    frame_count += 1

cap.release()
print("Done.")

Frame enhancement

In [ ]:
frame_path = sorted([os.path.join('../frames', f) for f in os.listdir('../frames') if f.endswith(".jpg")])

if not os.path.exists('../enhanced'):
    os.makedirs('../enhanced')

idx = 0
for f_path in frame_path:
    img = cv2.imread(f_path)

    # 灰度化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 提高对比度（CLAHE）
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)

    # 锐化
    blurred = cv2.GaussianBlur(enhanced, (7, 7), 10.0)
    sharpened = cv2.addWeighted(enhanced, 2, blurred, -0.5, 0)

    filename = os.path.join('../enhanced', f"enhanced_frame_{idx:03d}.jpg")
    cv2.imwrite(filename, sharpened)
    cv2.destroyAllWindows()

    print(f"Processed {filename}")
    idx += 1

print("Done.")

> Here we introduce manual image selection and reshape for the following OCR step.

OCR

Model: PaddleOCR

In [ ]:
def ocr_to_dataframe(img_path, y_thresh=10):
    ocr = PaddleOCR(use_angle_cls=True, lang='ch')
    result = ocr.ocr(img_path, cls=True)

    if not result or result[0] is None:
        print(f"[!] No OCR result for {img_path}")
        return None

    boxes = result[0]
    data = []
    for box in boxes:
        (x0, y0), (x1, y1), (x2, y2), (x3, y3) = box[0]
        text = box[1][0]
        conf = box[1][1]
        x_center = (x0 + x2) / 2
        y_center = (y0 + y2) / 2
        data.append((text, conf, x_center, y_center))

    data = sorted(data, key=lambda x: x[3])

    rows = []
    current_row = []
    for i, item in enumerate(data):
        if i == 0:
            current_row.append(item)
            continue
        _, _, _, y = item
        _, _, _, prev_y = data[i - 1]

        if abs(y - prev_y) > y_thresh:
            rows.append(current_row)
            current_row = [item]
        else:
            current_row.append(item)
    if current_row:
        rows.append(current_row)

    final_table = []
    for row in rows:
        sorted_row = sorted(row, key=lambda x: x[2])
        texts = [cell[0] for cell in sorted_row]
        final_table.append(texts)

    return pd.DataFrame(final_table)

Implement

In [ ]:
img_dir = "../imgs"
img_paths = sorted([os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(".jpg")])

dfs = []
for idx, img_path in enumerate(img_paths):
    df = ocr_to_dataframe(img_path)
    if df is not None:
        dfs.append(df)
        print("df shape: ", df.shape, "\n")
        print(f"[✓] OCR Done: {img_path}")
    else:
        print(f"[!] Skipped: {img_path}")

print("Done.")

Concat and Output

In [ ]:
max_rows = max(df.shape[0] for df in dfs)
dfs = [df.reindex(index=range(max_rows)) for df in dfs]
merged_df = pd.concat(dfs, axis=1)

output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)

merged_df.to_csv(os.path.join(output_dir, "ocr_table.csv"), index=False, header=False)
print("Done.")